**Import dependencies**

In [1]:
# yahoo finance api
import yfinance as yf

# change point detection models
import ruptures as rpt

# standard libraries
from timeit import default_timer as timer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**0. Pre-Processing**

In [26]:
# get stock price 
def preprocessing(stock_name, start_date, end_date):
    
    '''
    1. Input stock name and time window.
    2. Return historical closing price as list for given time window.
    '''

    # assign company
    data = yf.Ticker(stock_name)
    
    # get historical stock data during given time window
    df = data.history(interval = '1d', start = start_date, end = end_date)

    # convert df column to array
    series_array = np.array(round(df['Close'], 4))
    
    # convert df column to list
    column = round(df['Close'], 4)

    return series_array, column

In [32]:
def get_predicted_change_points(stock_name, start_date, end_date, search_method, n_change_points):
    '''
    1. Input stock name, time window, search method and number of change points to detect.
    2. Return index of predicted change points.
    '''
    # number of change points to detect
    n_breaks = n_change_points

    # get stock data of given time window as series and df column
    series, column = preprocessing(stock_name, start_date, end_date)

    # get predicted change points by index
    model = search_method(model="l1").fit(series)
    change_points = model.predict(n_bkps=n_breaks-1)

    return change_points

In [34]:
def mean_squared_error(annotated_change_points, predicted_change_points):
    '''
    1. Input human-labeled change points and predicted change points.
    2. Return mean squared error between human-labeled change points and predicted change points.
    '''
    y = annotated_change_points
    y_bar = predicted_change_points
    
    # variable to store the summation of differences
    summation = 0 
    
    # find total number of items in list
    n = len(y) 
    
    # calculate mse values
    for i in range (0,n):  
        difference = y[i] - y_bar[i]  
        squared_difference = difference**2  
        summation = summation + squared_difference
    
    mse = summation/n
    
    return mse

**1. Change Point Precision**

In [36]:
predicted_change_points = get_predicted_change_points('TSLA', '2018-04-01', '2021-04-01', rpt.Binseg, 3)
predicted_change_points

[600, 670, 756]

In [37]:
annotated_change_points = [550, 600, 700]

In [38]:
mean_squared_error(annotated_change_points, predicted_change_points)

3512.0

In [29]:
change_points_converted = []
for i in change_points:
    change_points_converted.append(column.index[i-1])
change_points_converted = pd.to_datetime(change_points_converted)
change_points_converted

DatetimeIndex(['2020-01-21', '2020-08-17', '2020-11-24', '2021-03-31'], dtype='datetime64[ns]', freq=None)